In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph json dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

In [36]:
# 영어 프롬프트 + 출처 링크 요구 
prompt_test_eg_link = """

# Instructions
When the user requests information about a specific wedding-related company, follow these steps:

    Use web search to investigate customer reviews and the reputation of the mentioned wedding-related company. Refer to the following search queries:

        '[Company Name] reviews'
        '[Company Name] ratings'
        '[Company Name] customer feedback'
        '[Company Name] experiences'

# Results
1. Answer in Korean
2. Summarize the information gathered based on the following criteria and include web links to all sources of the each provided information :

    Overall reputation: The ratio of positive to negative customer reviews.
    Strengths: Frequently praised services or qualities mentioned by customers.
    Weaknesses: Complaints or areas for improvement mentioned by customers.
    Features: Unique services or benefits offered by the company.

3. Deliver the summarized information to the user in a clear and easy-to-understand manner. If necessary, provide additional tips related to wedding preparation.


# Start!

Previous conversation history:
{chat_history}

New input: {input}

"""


In [38]:
# 영어 프롬프트 + 출처 요구 X
prompt_test_eg = """

# Instructions
When the user requests information about a specific wedding-related company, follow these steps:

    Use web search to investigate customer reviews and the reputation of the mentioned wedding-related company. Refer to the following search queries:

        '[Company Name] reviews'
        '[Company Name] ratings'
        '[Company Name] customer feedback'
        '[Company Name] experiences'

# Results
1. Answer in Korean
2. Summarize the information gathered based on the following criteria :

    Overall reputation: The ratio of positive to negative customer reviews.
    Strengths: Frequently praised services or qualities mentioned by customers.
    Weaknesses: Complaints or areas for improvement mentioned by customers.
    Features: Unique services or benefits offered by the company.

3. Deliver the summarized information to the user in a clear and easy-to-understand manner. If necessary, provide additional tips related to wedding preparation.


# Start!

Previous conversation history:
{chat_history}

New input: {input}

"""


In [43]:
# 한국어 프롬프트 + 출처 링크 요구 
prompt_test_kor_link = """

# 지시 사항 
사용자가 특정 결혼 관련 업체에 대해 정보를 요청하면, 다음 단계를 수행하세요:

    웹 검색을 사용하여 사용자가 언급한 결혼 관련 업체의 고객 후기 및 평판을 조사하세요. 아래의 검색 쿼리를 참고하세요:

        '[업체 이름] 후기'
        '[업체 이름] 리뷰'
        '[업체 이름] 평점'
        '[업체 이름] 고객 경험'

# 결과 
조사한 정보를 다음 기준에 따라 요약하고, 제공된 모든 정보의 출처 웹 링크를 명확히 표시하세요 :

    전반적인 평판: 고객의 긍정적 또는 부정적 평가 비율.
    강점: 고객들이 자주 언급한 칭찬받는 서비스나 장점.
    약점: 고객들이 불만을 제기한 사항이나 개선이 필요한 부분.
    특징: 해당 업체만의 독특한 서비스나 혜택.
    
요약된 정보를 사용자에게 이해하기 쉽게 전달하세요. 필요한 경우, 결혼 준비와 관련한 추가 팁도 간단히 제공하세요.


# 시작!

Previous conversation history:
{chat_history}

New input: {input}


"""

In [40]:
# 기본 테스트용
prompt_test = """

# 시작!

Previous conversation history:
{chat_history}

New input: {input}


"""

In [44]:
import json

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


class ReviewAssistant:
    def __init__(self):
        """
        리뷰 어시스턴트 초기화
        
        Args:
            budget_data (dict): 사용자의 예산 정보
        """
        
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            input_key="input"
        )
        
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "input"],
            template=prompt_test_kor_link  ##
        )
        
        self.llm = ChatOpenAI(
            model="gpt-4o", #"gpt-4o-mini"
            temperature=0.2
        )
        
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )
    
    def ask(self, question):
        """
        질문에 대한 응답 생성
        
        Args:
            question (str): 사용자 질문
        
        Returns:
            str: LLM의 응답
        """
        return self.chain.run(
            input=question

        )

def main():
    
    assistant = ReviewAssistant()
    
    print("리뷰 정리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)")
    
    while True:
        user_input = input("\n질문: ")
        
        if user_input.lower() == 'quit':
            print("상담을 종료합니다. 감사합니다!")
            break
            
        try:
            response = assistant.ask(user_input)
            print("\n답변:", response)
        except Exception as e:
            print(f"\n오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

리뷰 정리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)

답변: 신라호텔 웨딩홀에 대한 고객 후기 및 평판을 조사한 결과를 요약하여 제공하겠습니다.

### 전반적인 평판
신라호텔 웨딩홀은 대체로 긍정적인 평가를 받고 있습니다. 많은 고객들이 고급스러운 분위기와 세심한 서비스에 만족하고 있으며, 평점도 높은 편입니다.

### 강점
1. **고급스러운 분위기**: 신라호텔은 고급스러운 인테리어와 우아한 분위기로 유명합니다. 많은 고객들이 결혼식 장소로서의 품격을 높게 평가하고 있습니다.
2. **세심한 서비스**: 직원들의 친절하고 세심한 서비스가 자주 언급됩니다. 고객들은 결혼식 당일의 원활한 진행과 세심한 배려에 만족하고 있습니다.
3. **맛있는 음식**: 신라호텔의 음식은 높은 품질과 맛으로 칭찬받고 있습니다. 다양한 메뉴 선택과 맛있는 요리가 결혼식의 만족도를 높이고 있습니다.

### 약점
1. **비용**: 일부 고객들은 높은 비용을 단점으로 언급합니다. 신라호텔의 웨딩 패키지는 고급스러운 만큼 가격이 높은 편입니다.
2. **예약 경쟁**: 인기 있는 장소인 만큼 예약이 어렵다는 의견도 있습니다. 원하는 날짜에 예약을 하기 위해서는 미리 준비가 필요합니다.

### 특징
- **전문 웨딩 플래너**: 신라호텔은 전문 웨딩 플래너가 배정되어 결혼식 준비를 도와줍니다. 고객의 요구에 맞춘 맞춤형 서비스를 제공하여 특별한 결혼식을 계획할 수 있습니다.
- **다양한 웨딩 패키지**: 다양한 웨딩 패키지를 제공하여 고객의 예산과 스타일에 맞는 선택이 가능합니다.

### 추가 팁
결혼식을 준비할 때는 예산을 미리 설정하고, 원하는 날짜에 맞춰 일정을 조율하는 것이 중요합니다. 또한, 여러 업체의 후기를 비교하여 자신에게 가장 적합한 장소를 선택하는 것이 좋습니다.

위의 정보는 여러 웹사이트의 고객 후기와 평판을 바탕으로 요약한 것입니다. 추가적인 정보가 필요하시면, 직접 웹 검색을 통해 더 많은 후기를 확인해 보시기 바랍니다.



### 결과

신라호텔 웨딩홀 후기 알려줘 -> 답변 잘하고 확인해본 결과 내용도 꽤 정확함, 하지만 출처 링크는 잘못된 링크를 알려줌, 여러 번 해봤는데 잘 안되어서 출처 링크 필요하면 tavily 사용이 어떨지

요즘 인기있는 웨딩홀 후기 알려줘 -> 실제로 있는 웨딩홀에 대해 잘 대답하는 것으로 봐선 내부적으로 웹 검색이 잘 작동되긴 하는 것 같음

신라호텔 웨딩홀 후기 알려줘, 웹 검색을 사용해서 검색 결과를 잘 정리해주고 출처 링크도 알려줘 -> 잘못된 출처 링크 알려줌